In [1]:

from collections import Counter
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Load the iris dataset
iris = load_iris()
X, y = iris.data, iris.target


In [2]:
class RandomForestClassifier:
    def __init__(self, n_estimators: int = 10, max_depth: int = 5, min_samples_leaf: int = 3, max_features: int | None = None):
        self.n_estimators = n_estimators # Number of decision trees
        self.max_depth = max_depth # Max depth of each decision tree
        self.min_samples_leaf = min_samples_leaf # Min samples for a node to be declared as a leaf
        self.max_features = max_features # Optional input which restricts the max number of features a tree can be trained on
        self.trees: list = [] # Stores the trees and corresponding features used


    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        '''
        Trains the random forest by training multiple decision trees.

        For each decision tree, choose a random sample of data points
        and a random set of features to train it on. This ensures that
        no feature is dominant and reduces overfitting. The samples
        shared between trees can overlap (choosing with replacement).
        '''

        self.trees = [] # Initializing an empty list
        n_samples, n_features = X.shape
        self.max_features = self.max_features or int(np.sqrt(n_features)) # If no value is given, reverts to default (sqrt)

        for _ in range(self.n_estimators):
            # TODO 1: Take random samples of the dataset from here (Bootstrapping)
            indices = np.random.choice(n_samples, n_samples, replace=True)
            X_sample, y_sample = X[indices], y[indices]

            tree = DecisionTreeClassifier(max_depth=self.max_depth, min_samples_leaf=self.min_samples_leaf)
            # TODO 2: Train the decision tree using random features
            # YOUR CODE HERE
            tree_features = np.random.choice(n_features, self.max_features, replace=False)
            X_sample_subfeatures = X_sample[:, tree_features]
            tree = DecisionTreeClassifier(max_depth=self.max_depth, min_samples_leaf=self.min_samples_leaf)
            tree.fit(X_sample_subfeatures, y_sample)
            self.trees.append((tree, tree_features))

            

    def predict(self, X: np.ndarray) -> np.ndarray:
        '''
        Predicts class labels for the given input data.

        Pass the given data through all decision trees in self.trees,
        then choose the class that is given by majority of the trees.
        Use only the features that the tree was trained on -
        (the feature indices are also stored in the list self.trees)
        '''
        # TODO 3: Complete the predict function
        # YOUR CODE HERE
        predictions = []

        for tree, tree_features in self.trees:
            predictions.append(tree.predict(X[:, tree_features]))
        predictions = np.array(predictions).T  
        majority_vote = [Counter(sample_predictions).most_common(1)[0][0] for sample_predictions in predictions]
        return np.array(majority_vote)
        

In [3]:

# Instantiate and use the RandomForestClassifier correctly
model = RandomForestClassifier(n_estimators=10, max_depth=5, min_samples_leaf=3, max_features=None)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Predict and calculate accuracy
y_pred = model.predict(X_test)
accuracy = np.mean(y_pred == y_test)
print(f"Your model has an accuracy of {100 * accuracy:.2f}%.")


Your model has an accuracy of 96.67%.


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = np.mean(y_pred == y_test)
print(f"Your model has an accuracy of {100 * accuracy}%.")

Your model has an accuracy of 100.0%.
